In [1]:
from transformers import AutoTokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from torch.utils.data import Dataset
from pjs.dataset import Data
import pandas as pd
from loguru import logger

c:\Users\laure\AppData\Local\pypoetry\Cache\virtualenvs\pjs-8TwGqzJm-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = Data(data_path="../data/LMentry")

2025-04-16 00:46:00.875 | INFO     | pjs.dataset:collect_data:41 - Successfully loaded 'examples' from: ..\data\LMentry\bigger_number.json
2025-04-16 00:46:00.886 | INFO     | pjs.dataset:collect_data:41 - Successfully loaded 'examples' from: ..\data\LMentry\first_alphabetically.json
2025-04-16 00:46:00.894 | INFO     | pjs.dataset:collect_data:41 - Successfully loaded 'examples' from: ..\data\LMentry\first_alphabetically_consecutive_first_letter.json
2025-04-16 00:46:00.904 | INFO     | pjs.dataset:collect_data:41 - Successfully loaded 'examples' from: ..\data\LMentry\first_alphabetically_different_first_letter.json
2025-04-16 00:46:00.914 | INFO     | pjs.dataset:collect_data:41 - Successfully loaded 'examples' from: ..\data\LMentry\first_alphabetically_far_first_letter.json
2025-04-16 00:46:00.923 | INFO     | pjs.dataset:collect_data:41 - Successfully loaded 'examples' from: ..\data\LMentry\first_alphabetically_same_first_letter.json
2025-04-16 00:46:00.931 | INFO     | pjs.dataset

In [3]:
data.split(0.8)

2025-04-16 00:46:01.229 | INFO     | pjs.dataset:split:63 - Split for bigger_number.json: 2400 train / 600 test rows
2025-04-16 00:46:01.230 | INFO     | pjs.dataset:split:63 - Split for first_alphabetically.json: 2400 train / 600 test rows
2025-04-16 00:46:01.231 | INFO     | pjs.dataset:split:63 - Split for first_alphabetically_consecutive_first_letter.json: 2400 train / 600 test rows
2025-04-16 00:46:01.233 | INFO     | pjs.dataset:split:63 - Split for first_alphabetically_different_first_letter.json: 2400 train / 600 test rows
2025-04-16 00:46:01.234 | INFO     | pjs.dataset:split:63 - Split for first_alphabetically_far_first_letter.json: 2400 train / 600 test rows
2025-04-16 00:46:01.235 | INFO     | pjs.dataset:split:63 - Split for first_alphabetically_same_first_letter.json: 2400 train / 600 test rows
2025-04-16 00:46:01.236 | INFO     | pjs.dataset:split:63 - Split for first_letter.json: 2400 train / 600 test rows
2025-04-16 00:46:01.237 | INFO     | pjs.dataset:split:63 - Spli

In [4]:
data.save_dataset_state("../data/dataset_splits")

2025-04-16 00:46:01.265 | INFO     | pjs.dataset:save_dataset_state:94 - Saved train set to ../data/dataset_splits\split_20250416_004601\bigger_number_train.json
2025-04-16 00:46:01.268 | INFO     | pjs.dataset:save_dataset_state:94 - Saved test set to ../data/dataset_splits\split_20250416_004601\bigger_number_test.json
2025-04-16 00:46:01.278 | INFO     | pjs.dataset:save_dataset_state:94 - Saved train set to ../data/dataset_splits\split_20250416_004601\first_alphabetically_train.json
2025-04-16 00:46:01.556 | INFO     | pjs.dataset:save_dataset_state:94 - Saved test set to ../data/dataset_splits\split_20250416_004601\first_alphabetically_test.json
2025-04-16 00:46:01.568 | INFO     | pjs.dataset:save_dataset_state:94 - Saved train set to ../data/dataset_splits\split_20250416_004601\first_alphabetically_consecutive_first_letter_train.json
2025-04-16 00:46:01.571 | INFO     | pjs.dataset:save_dataset_state:94 - Saved test set to ../data/dataset_splits\split_20250416_004601\first_alphab

In [5]:
model_name = "phonemetransformers/GPT2-85M-CHAR-TXT"

## Macht W's statt leerzeichen, vielleicht lieber BABYLM-TOKENIZER-CHAR-TXT-SPACELESS ?
tokenizer = AutoTokenizer.from_pretrained('phonemetransformers/babble-tokenizers', subfolder='BABYLM-TOKENIZER-CHAR-TXT')

model = GPT2LMHeadModel.from_pretrained(model_name)

In [6]:
input_text = "Q: In an alphabetical order, which word comes first, \"blanket\" or \"diary\"? A:"
tokens = tokenizer.tokenize(input_text)
logger.info(tokens)

2025-04-16 00:46:04.126 | INFO     | __main__:<module>:3 - ['q', ':', 'W', 'i', 'n', 'W', 'a', 'n', 'W', 'a', 'l', 'p', 'h', 'a', 'b', 'e', 't', 'i', 'c', 'a', 'l', 'W', 'o', 'r', 'd', 'e', 'r', ',', 'W', 'w', 'h', 'i', 'c', 'h', 'W', 'w', 'o', 'r', 'd', 'W', 'c', 'o', 'm', 'e', 's', 'W', 'f', 'i', 'r', 's', 't', ',', 'W', '"', 'b', 'l', 'a', 'n', 'k', 'e', 't', '"', 'W', 'o', 'r', 'W', '"', 'd', 'i', 'a', 'r', 'y', '"', '?', 'W', 'a', ':']


In [7]:
encodings = data.get_tokenised_train_split(tokenizer)

input_encodings = encodings["first_word.json"][0]
label_encodings = encodings["first_word.json"][1]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [8]:
class CustomDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        return {
            key: val[idx] for key, val in self.encodings.items()
        }

encodings = {
    "input_ids": input_encodings["input_ids"],
    "attention_mask": input_encodings["attention_mask"],
    "labels": label_encodings["input_ids"],
}

dataset = CustomDataset(encodings)

training_args = TrainingArguments(
    output_dir="../data/results",
    overwrite_output_dir=True,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    save_steps=500,
    save_total_limit=2,
    logging_steps=100,
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()


C:\Users\laure\AppData\Local\Temp\ipykernel_20996\1051673243.py:33: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,0.677600
200,0.432000
300,0.400500
400,0.383500
500,0.380300
600,0.380000
700,0.362600
800,0.361300
900,0.368900


TrainOutput(global_step=900, training_loss=0.4162959416707357, metrics={'train_runtime': 986.6348, 'train_samples_per_second': 7.298, 'train_steps_per_second': 0.912, 'total_flos': 422558208000000.0, 'train_loss': 0.4162959416707357, 'epoch': 3.0})

In [9]:
test_data = data.split_data["first_word.json"]["test"]

logger.info(test_data)

2025-04-16 01:02:46.972 | INFO     | __main__:<module>:3 -                                                  input output
0    Q: In the sentence "John was lots more obnoxio...   John
1    Q: In the sentence "A boy hit the ball", what ...      A
2    Q: In the sentence "I remembered having kissed...      I
3    Q: What is the first word of the sentence "I b...      I
4    Write the first word of the sentence "John wil...   John
..                                                 ...    ...
595  Q: What is the first word of the sentence "Sus...  Susan
596  Write the first word of the sentence "John mad...   John
597  Q: What is the first word of the sentence "We ...     We
598  Q: In the sentence "John scratched his arm and...   John
599  Q: In the sentence "The brave are not afraid t...    The

[600 rows x 2 columns]


In [15]:
for index, row in test_data.head(2).iterrows():
    print(row["input"], row["output"])
    input = row["input"]

    inputs = tokenizer(
    input,
    return_tensors='pt',
    padding=True,
    truncation=True,
    max_length=200,
    return_attention_mask=True
    )
    
    # Modell generiert Fortsetzung
    outputs = model.generate(
    input_ids=inputs['input_ids'],
    attention_mask=inputs['attention_mask'],
    max_length=150,
    pad_token_id=tokenizer.eos_token_id
    )
    
    # Ausgabe-Text
    output_text = tokenizer.decode(outputs[0])
    output_text = output_text.replace("UTT_BOUNDARY", "")
    output_text = output_text.replace(" ", "")
    output_text = output_text.replace("W", " ")
    
    print(f"Input: {row["input"]}")
    print(f"Expected: {row['output']}")
    print(f"Predicted: {output_text}")
    print("---")

Q: In the sentence "John was lots more obnoxious than Fred", what is the first word? A: John
Input: Q: In the sentence "John was lots more obnoxious than Fred", what is the first word? A:
Expected: John
Predicted: q: in the sentence "john was lots more obnoxious than fred", what is the first word? a: hat is the first word? a: is the first woringlerififingfisfin
---
Q: In the sentence "A boy hit the ball", what is the first word? A: A
Input: Q: In the sentence "A boy hit the ball", what is the first word? A:
Expected: A
Predicted: q: in the sentence "a boy hit the ball", what is the first word? a: ell, i think that was a good idea.
---
